# Graph Convolutional Network by Kipf and Welling

## Imports

In [35]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

import pickle
import numpy as np

import itertools

import Notebooks.performance as pf

from scipy.spatial import distance

## GNN Definition

In [2]:
class LinearModule(nn.Module):
    """The linear transformation part of the GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(LinearModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation # This is the activation function

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

In [3]:
class GCN(nn.Module):
    """A GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = LinearModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(message_func=fn.copy_src(src='h', out='m'), reduce_func=fn.sum(msg='m', out='h'))
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

In [4]:
class Net(nn.Module):
    def __init__(self, infeats, hidden_size, outfeats):
        super(Net, self).__init__()
        self.gcn1 = GCN(infeats, hidden_size, F.relu)
        self.gcn2 = GCN(hidden_size, hidden_size, F.relu)
        self.gcn3 = GCN(hidden_size, outfeats, F.relu)
        self.dropout = nn.Dropout(0.2)
        

    def forward(self, g, features):
        x = self.gcn1(g, features)
        #x = self.dropout(x)
        #x = self.gcn2(g, x)
        x = self.gcn3(g, x)
        #x = F.log_softmax(x,1)
        return x

In [149]:
pairg = g.to_networkx()
additional_edges = []
while len(additional_edges)<10000:
    e = (np.random.randint(0,pairg.number_of_nodes()),np.random.randint(0,pairg.number_of_nodes()))
    if not (e in pairg.edges): additional_edges.append(e)
pairg.add_edges_from(additional_edges)
pairsample = (th.LongTensor([i for (i,j) in pairg.edges()]),th.LongTensor([j for (i,j) in pairg.edges()]))
pairsample[0].shape

torch.Size([23262])

In [150]:
adj_index = th.Tensor(nx.to_numpy_matrix(pairg)).bool().view(-1)

In [152]:
class PairNet(nn.Module):
    def __init__(self, infeats, hidden_size, outfeats):
        super(PairNet, self).__init__()
        self.Net = Net(infeats, hidden_size, outfeats)
        self.linear = nn.Linear(outfeats*2, 20)
        self.linear2 = nn.Linear(20, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, g, features):
        x = self.Net(g, features)
        #x_rep = x.repeat_interleave(x.shape[0],dim=0)
        x_rep=th.index_select(x,dim=0,index=pairsample[0])
        #y_rep = x.repeat(x.shape[0],1)
        y_rep=th.index_select(x,dim=0,index=pairsample[1])
        comb = th.cat([x_rep,y_rep],dim=1)
        result = self.dropout(comb)
        result = self.linear(result)
        result = F.relu(result)
        result = self.dropout(result)
        result = self.linear2(result)
        return result

In [153]:
def pairall(thenet, g, features):
        sig = nn.Sigmoid()
        x = thenet.Net(g, features)
        x_rep = x.repeat_interleave(x.shape[0],dim=0)
        y_rep = x.repeat(x.shape[0],1)
        comb = th.cat([x_rep,y_rep],dim=1)
        result = thenet.linear(comb)
        result = F.relu(result)
        result = thenet.linear2(result)
        result = sig(result)
        return result

In [86]:
net = PairNet(features.shape[1], 21, 4)
pred = net(g, features)
pred.shape

torch.Size([18264, 1])

In [106]:
pred

tensor([[0.5630],
        [0.5682],
        [0.5641],
        ...,
        [0.5630],
        [0.5620],
        [0.5615]], grad_fn=<SigmoidBackward>)

In [100]:
pred.view(features.shape[0],features.shape[0])

tensor([[0.5630, 0.5682, 0.5641,  ..., 0.5683, 0.5673, 0.5668],
        [0.5555, 0.5608, 0.5566,  ..., 0.5608, 0.5598, 0.5593],
        [0.5614, 0.5667, 0.5625,  ..., 0.5668, 0.5657, 0.5652],
        ...,
        [0.5555, 0.5608, 0.5566,  ..., 0.5608, 0.5598, 0.5593],
        [0.5568, 0.5621, 0.5579,  ..., 0.5621, 0.5611, 0.5606],
        [0.5577, 0.5630, 0.5588,  ..., 0.5630, 0.5620, 0.5615]],
       grad_fn=<ViewBackward>)

## Data Loading

In [156]:
from dgl.data import citation_graph as citegrh
import networkx as nx

data = citegrh.load_cora()
features = th.FloatTensor(data.features)
labels = th.LongTensor(data.labels)
mask = th.BoolTensor(data.train_mask)
g = data.graph

# add self loop
g.remove_edges_from(nx.selfloop_edges(g))
g = DGLGraph(g)
g.add_edges(g.nodes(), g.nodes())

In [157]:
labels_1hot = np.eye(np.max(labels.numpy()) + 1)[labels.numpy()]
labels_pair=th.FloatTensor(np.dot(labels_1hot,np.transpose(labels_1hot))).view(-1,1)
labels_pair.shape

torch.Size([7333264, 1])

## Select Training Set

In [13]:
percentage_train = 0.5

with open("data/cora_permutation1.pickle","rb") as f:
    perm1 = pickle.load(f)
mask = np.zeros(g.number_of_nodes())
mask[perm1[range(int(percentage_train*g.number_of_nodes()))]] = 1
mask = th.BoolTensor(mask)

In [83]:
features = g.in_degrees().float().unsqueeze(1)
citeseer_features = citeseer_g.in_degrees().float().unsqueeze(1)

In [84]:
features=th.cat([features,th.rand(size=(g.number_of_nodes(),1000))],1)

In [14]:
features=th.eye(g.number_of_nodes())

## Training

In [16]:
loss_function = nn.BCEWithLogitsLoss() #pf.perm_inv_loss(labels)

In [281]:
labels_pair[g.adjacency_matrix().to_dense().bool().view(-1),:].sum()/g.adjacency_matrix().to_dense().sum()

C:\Users\User\Anaconda3\lib\site-packages\dgl\base.py:25: UserWarning: Currently adjacency_matrix() returns a matrix with destination as rows by default.  In 0.5 the result will have source as rows (i.e. transpose=True)
  warnings.warn(msg, warn_type)


tensor(0.8488)

In [158]:
import time

net = PairNet(features.shape[1], 21, 21)
#print(net)

optimizer = th.optim.Adam(net.parameters(), lr=1e-2, weight_decay=0)
net.train() # Set to training mode (use dropout)

dur = []
for epoch in range(800):
    if epoch >=3:
        t0 = time.time()

    # Compute loss for test nodes (only for validation, not used by optimizer)
    #net.eval()
    #prediction = net(g, features)
    #train_rand=pf.rand_score(labels[mask].numpy(),np.argmax(prediction[mask].detach().numpy(), axis=1))
    #validation_rand=pf.rand_score(labels[~mask].numpy(),np.argmax(prediction[~mask].detach().numpy(), axis=1))
    train_rand=0
    validation_rand=0
    #net.train()

    # Compute loss for train nodes
    prediction = net(g, features)

    #loss = loss_function.approximate_loss(logits,mask,nclasses=3)
    
    #loss = F.nll_loss(logits[mask], labels[mask])
    loss = loss_function(prediction,labels_pair[adj_index,:])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)
        print(f"Epoch {epoch:05d} | Loss {loss.item():.4f} | Train.Rand {train_rand:.4f} | Valid.Rand {validation_rand:.4f} | Time(s) {np.mean(dur):.4f}")
    else:
        print(f"Epoch {epoch:05d} | Loss {loss.item():.4f} | Train.Rand {train_rand:.4f} | Valid.Rand {validation_rand:.4f} | Time(s) unknown")

ch 00559 | Loss 0.5179 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3366
Epoch 00560 | Loss 0.5194 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3365
Epoch 00561 | Loss 0.5184 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3366
Epoch 00562 | Loss 0.5193 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3366
Epoch 00563 | Loss 0.5191 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3366
Epoch 00564 | Loss 0.5219 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3366
Epoch 00565 | Loss 0.5174 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3366
Epoch 00566 | Loss 0.5200 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3366
Epoch 00567 | Loss 0.5206 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3366
Epoch 00568 | Loss 0.5180 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3366
Epoch 00569 | Loss 0.5184 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3366
Epoch 00570 | Loss 0.5163 | Train.Rand 0.0000 | Valid.Rand 0.0000 | Time(s) 0.3366
Epoch 0

In [159]:
net.eval()
final_prediction = np.round(pairall(net, g, features).detach().view(g.number_of_nodes(),g.number_of_nodes()))
loss_function(net(g, features),labels_pair[adj_index,:])

tensor(0.4760, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [122]:
labels_pair[adj_index,:].sum()/labels_pair[adj_index,:].shape[0]

tensor(0.6637)

In [125]:

print(f"Percentage of links in complete graph. Predicted: {(final_prediction.sum()/(g.number_of_nodes()**2)).item():.4f} | Real: {(labels_pair.sum()/(g.number_of_nodes()**2)).item():.4f}")

Percentage of links in complete graph. Predicted: 0.5108 | Real: 0.1796


In [148]:
final_prediction

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [127]:
final_prediction=labels_pair.view(2708,2708)

### Use connected components of similarity graph where bridgy links have been removed

In [143]:
def dissimmat(final_prediction):
    dissims = {(i,j): distance.jaccard(f1,f2) for i,f1 in enumerate(final_prediction) for j,f2 in enumerate(final_prediction) if j>=i}
    diss_matrix=np.array([[dissims[(i,j)] if (i,j) in dissims else dissims[(j,i)] for j in range(final_prediction.shape[0])] for i in range(final_prediction.shape[0])])
    return diss_matrix

In [144]:
dissmat = dissimmat(final_prediction)

In [145]:
compG = nx.from_numpy_matrix(dissmat<0.2,create_using=nx.Graph())
print(nx.number_connected_components(compG))

7


In [146]:
# Compute labels out of connected components
pred_labels = np.zeros(2708)
for i,c in enumerate(nx.connected_components(compG)):
    pred_labels[list(c)] = i

In [147]:
pf.rand_score(labels,pred_labels)

0.020572259069134742

In [142]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)
check_symmetric(final_prediction)

False

## Evaluation

In [41]:
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(g, features).detach()
pf.performance_as_df(labels,final_prediction,mask)

,All,Train,Test
Rand-Index,0.779452,0.982582,0.596059
Mutual Information,0.749836,0.974923,0.592846
Variation of Information,0.906211,0.089857,1.474745


### old stuff

In [495]:
net.eval() # Set net to evaluation mode (deactivates dropout)
sigf = nn.Sigmoid()
#final_prediction = np.round(sigf(net(g, features)).detach())
# change shape
final_prediction = final_prediction.view(g.number_of_nodes(),g.number_of_nodes()).numpy()
#final_prediction = labels_pair.view(g.number_of_nodes(),g.number_of_nodes()).numpy()
#final_prediction = graphmatrix.view(g.number_of_nodes(),g.number_of_nodes()).numpy()
compG = nx.from_numpy_matrix(final_prediction,create_using=nx.Graph())

def neighborhood_overlap(g, u, v):
    n_common_nbrs = len(set(nx.common_neighbors(g, u, v)))
    n_join_nbrs = g.degree(u) + g.degree(v) - n_common_nbrs - 2
    return n_common_nbrs / n_join_nbrs

In [498]:
print(nx.number_connected_components(compG))
bridge_like = [(u,v) for (u,v) in compG.edges() if neighborhood_overlap(compG, u, v)<0.9]
print(bridge_like)
#compG.remove_edges_from(bridge_like)
#print(nx.number_connected_components(compG))

1


KeyboardInterrupt: 